In [17]:
print("OK")

OK


In [5]:
!pip uninstall pinecone-client -y
!pip install "pinecone-client>=5.0.0"

Found existing installation: pinecone-client 5.0.1
Uninstalling pinecone-client-5.0.1:
  Successfully uninstalled pinecone-client-5.0.1
  Using cached pinecone_client-5.0.1-py3-none-any.whl.metadata (19 kB)
Using cached pinecone_client-5.0.1-py3-none-any.whl (244 kB)


In [6]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec
from langchain.vectorstores import Pinecone as LangchainPinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from pinecone import ServerlessSpec
import os

In [7]:
PINECONE_API_KEY = "pcsk_4jmtUP_Q3DBbnPppz5Hjk1y4khGFv5JDSb9wkUb6pWWUuQVgfNB91fAdZHnXcTaJyecjsd"
PINECONE_ENV = "us-east-1"

In [8]:
def load_pdf(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [9]:
extracted_data = load_pdf("data/")

In [10]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [11]:
text_chunks =text_split(extracted_data)
print ("length of my chunks:", len(text_chunks))

length of my chunks: 5859


In [12]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [13]:
embeddings = download_hugging_face_embeddings()

In [14]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [15]:
query_result = embeddings.embed_query("Hello world")
print("Length:", len(query_result))

Length: 384


In [16]:
query_result

[-0.03447723388671875,
 0.031023330986499786,
 0.006734934169799089,
 0.026108985766768456,
 -0.03936197608709335,
 -0.16030241549015045,
 0.06692397594451904,
 -0.0064414674416184425,
 -0.04745053872466087,
 0.014758801087737083,
 0.07087535411119461,
 0.055527523159980774,
 0.019193340092897415,
 -0.026251334697008133,
 -0.010109581984579563,
 -0.026940491050481796,
 0.02230747416615486,
 -0.022226689383387566,
 -0.1496925801038742,
 -0.017493082210421562,
 0.007676294539123774,
 0.054352324455976486,
 0.0032544718123972416,
 0.03172592446208,
 -0.08462141454219818,
 -0.029405992478132248,
 0.05159566551446915,
 0.04812410846352577,
 -0.0033147993963211775,
 -0.05827924981713295,
 0.041969384998083115,
 0.022210655733942986,
 0.1281888484954834,
 -0.022338971495628357,
 -0.011656305752694607,
 0.06292837858200073,
 -0.032876238226890564,
 -0.09122607856988907,
 -0.03117538057267666,
 0.05269947275519371,
 0.04703478887677193,
 -0.08420297503471375,
 -0.03005623258650303,
 -0.02074482

In [4]:
!pip install sentence_transformers

In [ ]:
os.environ["PINECONE_API_KEY"] = "" // add key here.

pc = Pinecone()

index_name = "medical-chatbot"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # Make sure this matches your embedding dimension
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'  # Adjust to your region
        )
    )

index = pc.Index(index_name)

# Create LangChain embeddings object
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Prepare texts and metadata
texts = [t.page_content for t in text_chunks]
metadatas = [{} for _ in texts]  # or use your own metadata

# Convert texts to embeddings
text_embeddings = embeddings.embed_documents(texts)

# Upsert to Pinecone manually
vectors = [
    {"id": str(i), "values": emb, "metadata": meta}
    for i, (emb, meta) in enumerate(zip(text_embeddings, metadatas))
]

index.upsert(vectors=vectors, batch_size=100)

# Now wrap the index with LangChain
docsearch = LangchainPinecone(
    index=index,
    embedding_function=embeddings.embed_query,
    text_key="text",  # You must include `"text"` in your metadata for search
)

Upserted vectors: 100%|██████████| 5859/5859 [01:10<00:00, 83.49it/s] 
